In [ ]:
# https://ymd_h.gitlab.io/ymd_blog/posts/gym_on_google_colab_with_gnwrapper/
# https://www.youtube.com/watch?v=dWmJ5CXSKdw
!apt update > /dev/null 2>&1
!apt install xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install --upgrade pip wheel==0.38.4 setuptools==65.5.1
!pip install gym==0.25.1 gym_super_mario_bros==7.3.0 nes_py==8.2.1 gym-notebook-wrapper==1.3.3 > /dev/null 2>&1
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 > /dev/null 2>&1
# Install stable baselines for RL stuff
!pip install stable-baselines3[extra]==1.8.0 > /dev/null 2>&1

In [ ]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gnwrapper

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = gnwrapper.Monitor(env, directory="./")

In [ ]:
done = True
for step in range(1000):
    if done:
        env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
env.display()
env.close()

# Preprocess the environment

In [ ]:
# Import Frane Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking
import matplotlib.pyplot as plt

In [ ]:
# 1. Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')
state = env.reset()
print(state.shape)
plt.imshow(state[0])
# 6. Wrap it for the colab monitor
# env = gnwrapper.Monitor(env, directory="./")

#Train the RL Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import os for file path management
import os
# Import PPO for algos
from stable_baselines3 import PPO
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, start_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.start_freq = start_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, f'mario_model_{self.n_calls+self.start_freq}')
            self.model.save(model_path)
        return True

DRIVE_DIR = './drive/My Drive/Projects/reinforcement-learning-examples/'
CHECKPOINT_DIR = os.path.join(DRIVE_DIR, 'train/')
LOG_DIR = os.path.join(DRIVE_DIR, 'logs/')
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, start_freq=0, save_path=CHECKPOINT_DIR)

In [ ]:
# This is the AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

In [ ]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=10000, callback=callback)

# Test it Out

In [ ]:
model = PPO.load(os.path.join(CHECKPOINT_DIR, 'mario_model_10000'))

In [ ]:
env = gnwrapper.Monitor(env, directory="./")
state = env.reset()

for _ in range(10000):
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
env.display()
env.close()